Use fancy machine learning to predict whether an article makes it into Nature/Science or PRL.

In [1]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../')

%matplotlib inline
import matplotlib.pyplot as plt 
import time
import numpy as np
import scipy as sp

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
from nltk.corpus import stopwords
s=stopwords.words('english') + ['']

# Machine learning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.externals import joblib

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [2]:
query = session.query(Article).filter(or_(Article.journal_ref.like('Physics Review Letters%'),
                                          Article.journal_ref.like('Phys. Rev. Lett.%'),
                                          Article.journal_ref.like('PRL%')))
abstractPRL = [x.abstract for x in query.all()]
titlePRL = [x.title for x in query.all()]

In [3]:
query = session.query(Article).filter(or_(Article.journal_ref.like('Nature%'),
                                          Article.journal_ref.like('Nat.%'),
                                          Article.journal_ref.like('Science%')))
abstractNatureScience = [x.abstract for x in query.all()]
titleNatureScience = [x.title for x in query.all()]

In [4]:
session.close_all()

In [5]:
# Train with all the data. We'll save the model for future use.

# indNatureScience = len(abstractNatureScience)*4/5
# indPRL = len(abstractPRL) - (len(abstractNatureScience) - indNatureScience)

train_abstract = abstractPRL + abstractNatureScience
# train_title = titlePRL[:indPRL] + titleNatureScience[:indNatureScience]
train_target = [0]*len(abstractPRL) + [1]*len(abstractNatureScience)
# train_target_names = ['PRL']*indPRL + ['Nature']*indNatureScience

# test_abstract = abstractPRL[indPRL:] + abstractNatureScience[indNatureScience:]
# test_title = titlePRL[indPRL:] + titleNatureScience[indNatureScience:]
# test_target = [0]*len(abstractPRL[indPRL:]) + [1]*len(abstractNatureScience[indNatureScience:])
# test_target_names = ['PRL', 'Nature/Science']

In [6]:
print len(abstractNatureScience)
print len(abstractPRL)

3005
13259


In [7]:
#SVC(kernel='linear') is good
text_abstract_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
                              ('tfidf', TfidfTransformer()),
                              ('clf', SVC(kernel='linear', probability=True))])
text_abstract_clf.fit(train_abstract, train_target)

Pipeline(steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, charset=None,
        charset_error=None, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), prep...ar', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [8]:
# Save model
joblib.dump(text_abstract_clf, 'text_abstract.pkl')

['text_abstract.pkl',
 'text_abstract.pkl_01.npy',
 'text_abstract.pkl_02.npy',
 'text_abstract.pkl_03.npy',
 'text_abstract.pkl_04.npy',
 'text_abstract.pkl_05.npy',
 'text_abstract.pkl_06.npy',
 'text_abstract.pkl_07.npy',
 'text_abstract.pkl_08.npy',
 'text_abstract.pkl_09.npy',
 'text_abstract.pkl_10.npy',
 'text_abstract.pkl_11.npy',
 'text_abstract.pkl_12.npy',
 'text_abstract.pkl_13.npy',
 'text_abstract.pkl_14.npy',
 'text_abstract.pkl_15.npy',
 'text_abstract.pkl_16.npy']

In [10]:
def inverseVectorizer(val):
    return (key for key, value in text_abstract_clf.named_steps['vect'].vocabulary_.iteritems() if value == val).next()

# This is super inefficient!!!
sorted_coefs = sorted( ((i,v) for i, v in np.ndenumerate(text_abstract_clf.named_steps['clf'].coef_.todense()) ),
                      key=lambda x: x[1] )

print "Top 50 indicators of PRL:"
bottom = sorted_coefs[:50]
print ", ".join([ inverseVectorizer(item[0][1]) for item in bottom])
print ""
print "Top 50 indicators of Nature/Science:"
top = list(reversed(sorted_coefs[-50:]))
print ", ".join([ inverseVectorizer(item[0][1]) for item in top])

Top 50 indicators of PRL:
we, model, we propose, propose, of the, we study, behavior, study, calculations, simulations, mev, consider, to the, we consider, we discuss, letter, parameters, show, tensor, numerical, squeezing, condensate, in the presence, phys, we show that, present, is shown, show that, scheme, study the, investigated, method, we show, discuss, we derive, the corresponding, linear, edge, show how, find that, mesoscopic, this letter, we develop, derive, approximation, phys rev, find, analyze, shown, case

Top 50 indicators of Nature/Science:
here we, here, here we report, and, of, quantum, have, in, galaxies, been, high, has, materials, here we demonstrate, stars, devices, solar, fundamental, to, their, however, understanding, research, superconductivity, physics, radio, challenge, many, star, planet, science, these, that, electronic, stellar, pulsar, information, as, because, we report, universe, planets, matter, report, observations, or, here we show, yet, most, phenome